# Contextualized Automatic Scoring in STEM Education
# --------------------------------------------------
# Activity 1: Context-Based Science Assessment
# Activity 2: Peer Review and Contextualization

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch


In [ ]:

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:

# 1. Load and Preview the Dataset
# For demo: We'll create a tiny toy dataset.
# In class, replace this with: pd.read_csv('data/sample_datasets/science_responses.csv')
data = pd.DataFrame({
    "question": [
        "Explain what happens when ice melts.",
        "Why does a ball fall to the ground?",
        "Describe what causes rain.",
        "How does a plant make food?"
    ],
    "response": [
        "Ice turns into water because of heat.",
        "The ball falls because of gravity.",
        "Rain happens when clouds cool and water falls.",
        "A plant uses sunlight to make food by photosynthesis."
    ],
    "score": [1, 1, 1, 1],  # 1=Correct, 0=Incorrect (binary for demo)
})

print("Sample data:")
display(data)


In [ ]:

# 2. Preprocess Data
# We'll use the 'response' as input and 'score' as label.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['response'], padding=True, truncation=True, max_length=64)

# For a real dataset, split train/test.
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['response'].tolist(), data['score'].tolist(), test_size=0.25, random_state=42
)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)

class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = SimpleDataset(train_encodings, train_labels)
test_dataset = SimpleDataset(test_encodings, test_labels)


In [ ]:
# 3. Fine-tune BERT for Scoring (Binary Classification)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_strategy="epoch",
    disable_tqdm=False,
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

print("Training model (this may take a few minutes on CPU)...")
trainer.train()


In [ ]:
# 4. Use the Model for Automated Scoring

# Predict on test data
preds = trainer.predict(test_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)

print("Classification report:")
print(classification_report(test_labels, pred_labels))

In [ ]:
# 5. Activity: Try Your Own Response

def score_response(your_response):
    inputs = tokenizer([your_response], truncation=True, padding=True, max_length=64, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    if pred == 1:
        return "Correct / On Target"
    else:
        return "Needs Improvement"

print("\nTry scoring your own response:")
user_input = input("Enter a science answer to score (e.g., 'Plants use sunlight to make food.'):\n")
print("AI Score:", score_response(user_input))


In [ ]:
# 6. Activity 2: Peer Review and Contextualization

print("\nPeer Review Activity:")
print("1. In pairs or groups, exchange your written science responses.")
print("2. Each partner enters their peer's response and notes the AI's feedback.")
print("3. Compare the AI's score with your own judgment as a human reviewer.")
print("4. Discuss: Did the model miss context or nuance? How could the feedback be improved?")

## Reflection: Contextualization in AI Scoring

- Did the AI model give appropriate feedback?
- Were there cases where the model misunderstood the scientific context?
- How did the AI feedback compare to your peer's/human feedback?
- What could be improved in this scoring system?
- Why is understanding context important in STEM assessment?